In [ ]:
import logging

# Configurations
from configuration.llm_inference_configuration import APILLMConfiguration
from configuration.embedding_inference_configuration import APIEmbeddingModelConfiguration, LocalEmbeddingModelConfiguration
from configuration.nlp_configuration import NLPConfiguration
from configuration.tool_configuration import ToolConfiguration
from configuration.operator_configuration import CoTOperatorConfiguration
# System components
from llm.request_llm import RequestLLM
from embedding.local_embedding import LocalEmbeddingModel
from embedding.request_embedding import RequestEmbeddingModel
from nlp.spacy_nlp import SpacyNLP
from tools.python_code_runner import PythonCodeRunnerTool
from tools.demonstration_sampling import DemonstrationSamplingTool
# Prompt
from prompt.zero_shot import ZeroShotPrompt
from prompt.few_shot import FewShotPrompt
from prompt.user_message import UserMessagePrompt
from prompt.assistant_message import AssistantMessagePrompt
# Memory
from base_classes.memory.memory_atom import AbstractMemoryAtom
from base_classes.memory.memory_block import AbstractMemoryBlock
from base_classes.memory.memory_topic import AbstractMemoryTopic
from base_classes.memory.datatypes.data_item import PromptDataItem
from base_classes.memory.memory_worker import MemoryWorker
# MCP
# from tools.mcp_server
# Operator
from base_classes.operator import AbstractOperator
from operators.cot import CoTOperator
from operators.debate import DebateOperator
from operators.react import ReactOperator

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s\t\t%(levelname)s\t%(message)s',
    handlers=[logging.StreamHandler()]
)


# 1. System component test

## 1.1. LLM test

### 1.1.1. Azure OpenAI test

In [ ]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = RequestLLM(gpt_4o_azure_configuration)

try:
    azure_gpt_4o_1 = RequestLLM(gpt_4o_azure_configuration)
except Exception as e:
    logging.info("LLM unique ID test successful")
    logging.error(f"Failed to create AzureOpenAILLM instance: {e}")

In [ ]:
sample_zero_shot_message = ZeroShotPrompt(
    [
        {
            "role": "user", 
            "content": "Health check. Say 'hi' to start the conversation."
        }
    ]
)

sample_zero_shot_message_responses = azure_gpt_4o.query(query=sample_zero_shot_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_zero_shot_message_responses))

In [ ]:
sample_few_shots_message = FewShotPrompt(
    [
        {
            "role": "system", 
            "content": "You are NetMind assistant. You task is to answer to the user anything about Viettel Group."
        },
        {
            "role": "user", 
            "content": "Tell me about Viettel Group."
        },
        {
            "role": "assistant", 
            "content": "Viettel Group is a Vietnamese multinational telecommunications company headquartered in Hanoi, Vietnam. It is a state-owned enterprise and operated by the Ministry of Defence. You can find out more about Viettel Group at https://viettel.vn/."
        },
        {
            "role": "user",
            "content": "What is the revenue of Viettel Group? Tell me the list of child companies of Viettel Group?"
        }
    ]
)

sample_few_shots_message_responses = azure_gpt_4o.query(query=sample_few_shots_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_few_shots_message_responses))

## 1.2. Embedding test

### 1.2.1. Request embedding model test

In [ ]:
text_embedding_ada_002_config = APIEmbeddingModelConfiguration()
text_embedding_ada_002_config.load("configuration/yaml/embedding/text-embedding-ada-002.yaml")
text_embedding_ada_002 = RequestEmbeddingModel(text_embedding_ada_002_config)

In [ ]:
# text_embedding_ada_002.encode("Hello world")
text_embedding_ada_002.similarity("Hello world", "Hello world")

## 1.3. Tool test

### 1.3.1. Python code runner tool

In [ ]:
python_code_runner_config = ToolConfiguration()
python_code_runner_config.load("configuration/yaml/tools/python_code_runner.yaml")
python_code_runner = PythonCodeRunnerTool(python_code_runner_config)

In [ ]:
input_code_list = [
    {
        "code_string": "a = 11\nb = 20\nresult = a + b\n\ndef greet(name):\n return f'Hello, {name}!'\n \nmessage = greet('World')",
        "id": "1"
    },
    {
        "code_string": "def multiply(x, y):\n\treturn x * y\n\nnum1 = 5\nnum2 = 3\nresult = multiply(num1, num2)\n\nprint(f'The product of {num1} and {num2} is {result}.')",
        "id": "2"
    }
]

results = await python_code_runner.execute(input_code_list=input_code_list)
results

## 1.4. NLP model test 

### 1.4.1. SpaCy NLP test

In [ ]:
spacy_nlp_config = NLPConfiguration()
spacy_nlp_config.load("configuration/yaml/nlp/spacy.yaml")
spacy_model = SpacyNLP(spacy_nlp_config)

In [ ]:
spacy_model.lemmatize("Hello, my name is John Doe. I am a software engineer. I love coding and solving problems.")

# 2. Memory test

In [ ]:
# Create Memory Atoms
prompt_data_1 = [{
    'role': 'user',
    'content': 'Calculate the sum of 11 and 20.'
}]
mem_atom_1 = AbstractMemoryAtom(data=PromptDataItem(UserMessagePrompt(prompt_data_1)))

prompt_data_2 = [{
    'role': 'assistant',
    'content': 'In order to calculate the sum of two numbers, we need to do the following:\n\t1. Write a Python code with add_sum() function, receiving 2 variables.\n\t2. Execute the code with the given variables.\n\t3. Return the result of the sum.'
}]
mem_atom_2 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_2)))

prompt_data_3 = [{
    'role': 'assistant',
    'content': 'Here is the Python code to calculate the sum of two numbers:\n\na = 11\nb = 20\nresult = a + b\n\nprint(f"The sum of {a} and {b} is {result}.")'
}]
mem_atom_3 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_3)))

prompt_data_4 = [{
    'role': 'assistant',
    'content': 'After executing the code, we got the sum of 11 and 20 is 31.'
}]
mem_atom_4 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_4)))

# Create Memmory Block
mem_block = AbstractMemoryBlock()
mem_block.add_memory_atom(mem_atom_1)
mem_block.add_memory_atom(mem_atom_2)
mem_block.add_memory_atom(mem_atom_3)
mem_block.add_memory_atom(mem_atom_4)

mem_block.mem_atom_graph = {
        mem_atom_1.mem_atom_id: [mem_atom_2.mem_atom_id],
        mem_atom_2.mem_atom_id: [mem_atom_3.mem_atom_id, mem_atom_4.mem_atom_id],
        mem_atom_3.mem_atom_id: [],
        mem_atom_4.mem_atom_id: []
    }

# Create Memory Topic
mem_topic = AbstractMemoryTopic()
mem_topic.insert_mem_block(mem_block)
mem_topic.insert_mem_block(mem_block)
mem_topic.insert_mem_block(mem_block)
mem_topic.insert_mem_block(mem_block)

In [ ]:
# Memory Worker Initiation
spacy_nlp_config = NLPConfiguration()
spacy_nlp_config.load("configuration/yaml/nlp/spacy.yaml")
spacy_model = SpacyNLP(spacy_nlp_config)

gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = RequestLLM(gpt_4o_azure_configuration)

all_mini_v2_configuration = APIEmbeddingModelConfiguration()
all_mini_v2_configuration.load("configuration/yaml/embedding/all-MiniLM-L6-v2.yaml")
all_mini_v2 = RequestEmbeddingModel(all_mini_v2_configuration)

memory_worker = MemoryWorker(
    embedding_model=all_mini_v2,
    llm=azure_gpt_4o,
    nlp_model=spacy_model
)

# 3. Operator

## 3.1. CoT

In [ ]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = AzureOpenAILLM(gpt_4o_azure_configuration)

demonstration_sampling_tool_config = ToolConfiguration()
demonstration_sampling_tool_config.load("configuration/yaml/tools/demonstration_sampling.yaml")
demonstration_sampling_tool = DemonstrationSamplingTool(demonstration_sampling_tool_config)

In [ ]:
cot_operator_config = CoTOperatorConfiguration()
cot_operator_config.load("configuration/yaml/operators/cot.yaml")

In [ ]:
cot_operator = CoTOperator(config=cot_operator_config)

In [ ]:
CoTOperator.__dict__

In [ ]:
AzureOpenAILLM.__dict__['logger']